In [4]:
import chess
import chess.pgn
import numpy as np
import tensorflow as tf

In [5]:
# Data preprocessing functions...

# Parsing the PGN Data

def move_to_index(move):
    """Convert a move to an index in the range [0, 4095]."""
    source = move.from_square
    target = move.to_square
    return 64 * source + target


def extract_positions_and_moves_from_pgn(pgn_path, batch_size=10000):
    positions = []
    moves_list = []
    batch_num = 0
    
    with open(pgn_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            board = game.board()
            for move in game.mainline_moves():
                positions.append(board.fen())
                
                move_idx = move_to_index(move)
                one_hot_move = np.zeros(4096, dtype=np.int8)
                one_hot_move[move_idx] = 1
                moves_list.append(one_hot_move)
                
                board.push(move)
                
                # Check if the batch size is reached
                if len(positions) >= batch_size:
                    np.save(f'positions_batch_{batch_num}.npy', positions)
                    np.save(f'labels_batch_{batch_num}.npy', moves_list)
                    batch_num += 1
                    positions.clear()
                    moves_list.clear()
    
    # Save any remaining data
    if positions:
        np.save(f'positions_batch_{batch_num}.npy', positions)
        np.save(f'labels_batch_{batch_num}.npy', moves_list)

# Call the function
extract_positions_and_moves_from_pgn("Databases/lichess_db_standard_rated_2013-01.pgn")


OSError: Not enough free space to write 40960000 bytes

In [7]:
import numpy as np
import os

def fen_to_tensor(fen):
    fen = fen.split(' ')[0]  # Only take the piece placement data
    rows = fen.split('/')
    tensor = np.zeros((8, 8, 12), dtype=np.int8)

    for row_idx, row in enumerate(rows):
        col_idx = 0
        for char in row:
            if char.isdigit():
                col_idx += int(char)
            else:
                tensor[row_idx, col_idx, piece_to_index(char)] = 1
                col_idx += 1
    return tensor

def piece_to_index(piece):
    piece_dict = {
        'P': 0, 'p': 1,
        'N': 2, 'n': 3,
        'B': 4, 'b': 5,
        'R': 6, 'r': 7,
        'Q': 8, 'q': 9,
        'K': 10, 'k': 11
    }
    return piece_dict[piece]

batch_size = 10000
batch_num = 0

# Determine the total number of batches
total_batches = len([name for name in os.listdir('.') if name.startswith('positions_batch_')])

for batch_num in range(total_batches):
    # Load positions and labels for the current batch from disk
    batch_positions = np.load(f'positions_batch_{batch_num}.npy')
    batch_labels = np.load(f'labels_batch_{batch_num}.npy')
    
    batch_tensors = [fen_to_tensor(fen) for fen in batch_positions]
    
    np.save(f'tensors_batch_{batch_num}.npy', batch_tensors)
    
    del batch_positions  # Free up memory
    del batch_tensors
    del batch_labels

ValueError: cannot reshape array of size 0 into shape (10000,4096)

In [8]:
# Neural network model

model = tf.keras.models.Sequential()

# Convolutional layers
model.add(tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same', input_shape=(8, 8, 12)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding='same'))
model.add(tf.keras.layers.BatchNormalization())

# Fully connected layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation='relu'))

# Output layer
model.add(tf.keras.layers.Dense(768, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:


# Number of epochs
epochs = 10

# Total number of batches (assuming you know this from the preprocessing step)
total_batches = batch_num

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # Loop over all batches
    for i in range(total_batches):
        # Load tensors and labels for the current batch from disk
        batch_tensors = np.load(f'tensors_batch_{i}.npy')
        batch_labels = np.load(f'labels_batch_{i}.npy')  # Assuming you've saved labels in a similar manner
        
        # Train the model on the current batch
        model.train_on_batch(batch_tensors, batch_labels)
        
        # Optionally, you can print the training loss and accuracy for each batch
        loss, accuracy = model.evaluate(batch_tensors, batch_labels, verbose=0)
        print(f"Batch {i+1}/{total_batches} - loss: {loss:.4f} - accuracy: {accuracy:.4f}")
    
    # After each epoch, you can save the model or evaluate it on a validation set if you have one
    model.save(f"model_epoch_{epoch+1}.h5")


Epoch 1/10


ValueError: in user code:

    File "/home/guilherme/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/guilherme/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/guilherme/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/guilherme/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/guilherme/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/home/guilherme/.local/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 265, in

In [ ]:
# Training code...

def predict_move(board):
    board_tensor = preprocess_board(board)  # Convert the board to a tensor
    move_probabilities = model.predict(board_tensor)
    best_move_index = np.argmax(move_probabilities)
    return index_to_move(best_move_index)  # Convert the index back to a move


In [ ]:
# Chess-playing loop...
board = chess.Board()
while not board.is_game_over():
    move = predict_move(board)
    board.push(move)